In [104]:
import pandas as pd
import numpy as np
import re

In [105]:
import warnings
warnings.filterwarnings('ignore')

In [106]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [107]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [108]:
df = pd.read_csv("../data/IMDB Dataset.csv")
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [109]:
from preprocessing import to_lowercase, remove_html_tags, remove_punctuation

In [110]:
df["review"] = df["review"].apply(to_lowercase)
df["review"] = df["review"].apply(remove_html_tags)
df["review"] = df["review"].apply(remove_punctuation)

In [111]:
df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production the filming tec...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there s a family where a little boy ...,negative
4,petter mattei s love in the time of money is...,positive


In [112]:
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})
X = df["review"]
y = df["sentiment"]

In [113]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42, stratify=y_train)

In [114]:
MAX_FEATURES = 10000
MAX_LEN = 200
tokenizer = Tokenizer(num_words=MAX_FEATURES)
tokenizer.fit_on_texts(X_train)

In [115]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [116]:
X_train_padded = pad_sequences(X_train_seq, maxlen=MAX_LEN)
X_val_padded = pad_sequences(X_val_seq, maxlen=MAX_LEN)
X_test_padded = pad_sequences(X_test_seq, maxlen=MAX_LEN)

In [117]:
model = Sequential([
    Embedding(input_dim=MAX_FEATURES, output_dim=32, input_length=MAX_LEN),
    SimpleRNN(64),
    Dense(1, activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [118]:
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_8 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [119]:
model.fit(
    X_train_padded, y_train,
    epochs=5,
    batch_size=32,
    validation_data=(X_val_padded, y_val),
    verbose=1
)
score = model.evaluate(X_test_padded, y_test, verbose=0)
print(f"\nTest accuracy: {score[1]:.2f}")

Epoch 1/5
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 70s 57ms/step - accuracy: 0.6612 - loss: 0.5890 - val_accuracy: 0.7262 - val_loss: 0.5363
Epoch 2/5
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 60s 53ms/step - accuracy: 0.8354 - loss: 0.3800 - val_accuracy: 0.8380 - val_loss: 0.3898
Epoch 3/5
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 60s 53ms/step - accuracy: 0.7994 - loss: 0.4329 - val_accuracy: 0.8005 - val_loss: 0.4685
Epoch 4/5
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 60s 53ms/step - accuracy: 0.7891 - loss: 0.4495 - val_accuracy: 0.7865 - val_loss: 0.4898
Epoch 5/5
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 60s 53ms/step - accuracy: 0.8610 - loss: 0.3369 - val_accuracy: 0.6787 - val_loss: 0.5901

Test accuracy: 0.69


In [ ]:
def predict_sentiment(review_text):
    text = review_text.lower()
    text = re.sub(r"<.*?>", "", text)
    text = re.sub(r'[^a-z0-9\s]', '', text)

    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=MAX_LEN)

    prediction = model.predict(padded)[0][0]
    return f"{'Positive' if prediction >= 0.5 else 'Negative'} (Probability: {prediction:.2f})"


sample_review = "The food was great."
print(f"Review: {sample_review}")
print(f"Sentiment: {predict_sentiment(sample_review)}")

Review: The food was great.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
Sentiment: Positive (Probability: 0.77)


In [133]:
sample_review = "This is very good."
print(f"Review: {sample_review}")
print(f"Sentiment: {predict_sentiment(sample_review)}")

Review: This is very good.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
Sentiment: Positive (Probability: 0.76)
